# Purpose
The purpose of this notebook is to explore the usage of a Conversational Retrieval Agent from `langchain` for our use case.  The agent will allow the LLM to determine when to query the database and when not to.  I'm mainly working from the following examples: [example from documentation](https://python.langchain.com/docs/use_cases/question_answering/how_to/conversational_retrieval_agents), [blog post example](https://blog.langchain.dev/conversational-retrieval-agents/).

# Vector Store

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local("data/vector-db/", embeddings)

# Retriever Tool
We need to define a retriever tool for the LLM to use to retrieve context.  The name and description of the tool are important because they are used by the LLM to determine when to use the tool.

In [4]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    db.as_retriever(), 
    "search_finance_opinions",
    "Searches and returns documents regarding Reddit user's opinions on topics related to finance and investing."
)
tools = [tool]

# Agent

Now I need to build the agent using `create_conversational_retrieval_agent`, which just needs the tool provided above along with an LLM to use.  The agent uses `OpenAIFunctionsAgent` so I'll use an OpenAI chat model as the LLM.

In [5]:
import toml
import os

os.environ["OPENAI_API_KEY"] = toml.load("secrets.toml")["OPENAI_API_TOKEN"]

In [6]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

# Test Chat

Now I can test it out and monitor the decision making.

In [7]:
result = agent_executor({"input":"Hi my name is Coy and I'd like some advice."})
result



> Entering new AgentExecutor chain...
Hi Coy! I'm here to help. What kind of advice are you looking for?

> Finished chain.


{'input': "Hi my name is Coy and I'd like some advice.",
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False)],
 'output': "Hi Coy! I'm here to help. What kind of advice are you looking for?",
 'intermediate_steps': []}

In [8]:
result = agent_executor({"input":"I'd like to know what people are saying about Robinhood"})



> Entering new AgentExecutor chain...

Invoking: `search_finance_opinions` with `{'arg1': 'Robinhood'}`


[Document(page_content='CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they caused in losses. It might take a couple of days, but send Robinhood to the ground and GME to the moon./n/nChapman Albin is an investors rights firm that my buddy works at. Just got off the phone w him. He is going to post a press release regarding the case they are filing. \nLet me know if you need help finding a lawyer. \nDisclaimer: I’m not getting anything out of this', metadata={'id': 'l6x130'}), Document(page_content="Wallstreet Bets Set to Private Megathread/n/nThe moderators there

In [9]:
result

{'input': "I'd like to know what people are saying about Robinhood",
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False),
  HumanMessage(content="I'd like to know what people are saying about Robinhood", additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "arg1": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off u

In [10]:
result = agent_executor({"input":"So is it still safe to use?"})
result



> Entering new AgentExecutor chain...
The safety of using Robinhood or any other financial platform depends on various factors, including your personal risk tolerance and the specific features and security measures offered by the platform. It's important to conduct your own research and consider the following:

1. Security Measures: Look into the security measures implemented by Robinhood, such as two-factor authentication, encryption, and account protection features. Ensure that the platform has robust security protocols in place to protect your personal and financial information.

2. Regulatory Compliance: Check if Robinhood is regulated by relevant financial authorities in your country. Regulatory oversight can provide an additional layer of protection for users.

3. User Reviews and Reputation: Read reviews and feedback from other users to get an idea of their experiences with Robinhood. Consider both positive and negative reviews to make an informed decision.

4. Financial Stabi

{'input': 'So is it still safe to use?',
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False),
  HumanMessage(content="I'd like to know what people are saying about Robinhood", additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "arg1": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they ca